# Calibración de Probabilidades y Selección de Threshold

Este notebook forma parte del portafolio y ejemplifica la importancia de calibrar las probabilidades y ajustar el threshold en problemas de clasificación, especialmente cuando equivocarse es muy costoso.

## 1. Introducción
Contexto, objetivos y métricas de interés (precisión, recall, AUC, Brier score, costo).

## 2. Problema con alto costo por equivocación
Ejemplo: fraude, diagnóstico médico o rechazo de cliente valioso. Definir costos asimétricos para FN y FP.

## 3. Calibración de probabilidades
- Platt scaling (sigmoide)
- Isotonic regression
- Métricas de calibración (Brier score, reliability curve) y validación cruzada.

In [ ]:
# Setup inicial
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.metrics import (roc_auc_score, roc_curve, confusion_matrix, brier_score_loss,
                            precision_recall_fscore_support)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid", context="talk")

# Datos sintéticos
X, y = make_classification(n_samples=4000, n_features=20, n_informative=8, n_redundant=4,
                           weights=[0.9, 0.1], class_sep=1.0, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# Modelo base
base = LogisticRegression(max_iter=200, n_jobs=-1)
base.fit(X_train, y_train)
p_base = base.predict_proba(X_test)[:,1]
auc_base = roc_auc_score(y_test, p_base)
brier_base = brier_score_loss(y_test, p_base)
print(f'AUC base: {auc_base:.3f} | Brier base: {brier_base:.3f}')

# Calibración: Platt (sigmoide) e Isotónica
platt = CalibratedClassifierCV(base_estimator=LogisticRegression(max_iter=200), method='sigmoid', cv=5)
platt.fit(X_train, y_train)
p_platt = platt.predict_proba(X_test)[:,1]
auc_platt = roc_auc_score(y_test, p_platt); brier_platt = brier_score_loss(y_test, p_platt)

iso = CalibratedClassifierCV(base_estimator=LogisticRegression(max_iter=200), method='isotonic', cv=5)
iso.fit(X_train, y_train)
p_iso = iso.predict_proba(X_test)[:,1]
auc_iso = roc_auc_score(y_test, p_iso); brier_iso = brier_score_loss(y_test, p_iso)

print(f'AUC platt: {auc_platt:.3f} | Brier platt: {brier_platt:.3f}')
print(f'AUC iso:   {auc_iso:.3f} | Brier iso:   {brier_iso:.3f}')

# Curvas de calibración
def plot_reliability(y_true, p_list, labels):
    plt.figure(figsize=(7,6))
    for p, lab in zip(p_list, labels):
        frac_pos, mean_pred = calibration_curve(y_true, p, n_bins=10)
        plt.plot(mean_pred, frac_pos, marker='o', label=lab)
    plt.plot([0,1],[0,1],'k--', alpha=0.6)
    plt.xlabel('Probabilidad predicha promedio')
    plt.ylabel('Fracción positiva')
    plt.title('Curva de confiabilidad (calibración)')
    plt.legend(); plt.tight_layout(); plt.show()

plot_reliability(y_test, [p_base, p_platt, p_iso], ['Base','Platt','Isotónica'])


## 4. Experimentación con thresholds y costo
Definir una función de costo asimétrica y evaluar cómo cambia el costo total al variar el threshold. Mostrar matriz de confusión, ROC y curva de costo.

In [ ]:
def cost_metrics(y_true, p, thr, c_fp=1.0, c_fn=10.0):
    y_hat = (p >= thr).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, y_hat).ravel()
    total_cost = c_fp*fp + c_fn*fn
    return {'thr':thr, 'fp':int(fp), 'fn':int(fn), 'tp':int(tp), 'tn':int(tn), 'cost': total_cost}

thresholds = np.linspace(0.01, 0.99, 50)
rows = []
for t in thresholds:
    rows.append(cost_metrics(y_test, p_iso, t, c_fp=1.0, c_fn=10.0))
df_cost = pd.DataFrame(rows)
display(df_cost.head())

plt.figure(figsize=(7,5))
plt.plot(df_cost['thr'], df_cost['cost'], label='Costo total (Isotónica)')
plt.xlabel('Threshold'); plt.ylabel('Costo total'); plt.title('Curva de costo vs threshold')
plt.legend(); plt.tight_layout(); plt.show()

# Seleccionar threshold que minimiza costo
t_opt = df_cost.loc[df_cost['cost'].idxmin(), 'thr']
print(f'Threshold óptimo (mín costo): {t_opt:.3f}')


## 5. Conclusión
Resumen de hallazgos: la calibración mejora la confiabilidad de las probabilidades y permite una elección de threshold informada por el costo de error.